# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut
import os
deeplabcut.__version__

'2.0.4.1'

In [2]:
examples_dir = os.getcwd()
print(examples_dir)
# os.chdir('/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples')

/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples


In [3]:
task='Track_flies' # Enter the name of your experiment Task
experimenter='Sayed' # Enter the name of the experimenter
video=['/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy04_reshaped_top.avi'] # Enter the paths of your videos you want to grab frames from.

# path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory=examples_dir, copy_videos=True) #change the working directory to where you want the folders created.

# The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
path_config_file = "/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/config.yaml"

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [4]:
%matplotlib inline

deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=True, checkcropping=True, ) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/videos/outpy04_reshaped_top.avi ?

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

Working on folder: outpy04_reshaped_top
You can now check the labels, using 'check_labels' before proceeding. Then,  you can use the function 'create_training_dataset' to create the training dataset.


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [4]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Sayed.
They are stored in the following folder: /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/labeled-data/outpy04_reshaped_top_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [6]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training - If you want to use a CPU, continue. 
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(path_config_file, displayiters=100, saveiters=1000)

Config:
{'all_joints': [[0]],
 'all_joints_names': ['Fly1'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Track_fliesFeb20/Track_flies_Sayed95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Track_fliesFeb20/Documentation_data-Track_flies_95shuffle1.pickle',
 'minsize': 100,
 'mirror': False,
 'multi_step': [[0.00

INFO:tensorflow:Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt
Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt
Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Display_iters overwritten as 100
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/dlc-models/iteration-0/Track_fliesFeb20-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheigh

iteration: 100 loss: 0.0420 lr: 0.005
iteration: 100 loss: 0.0420 lr: 0.005
iteration: 100 loss: 0.0420 lr: 0.005
iteration: 200 loss: 0.0125 lr: 0.005
iteration: 200 loss: 0.0125 lr: 0.005
iteration: 200 loss: 0.0125 lr: 0.005
iteration: 300 loss: 0.0084 lr: 0.005
iteration: 300 loss: 0.0084 lr: 0.005
iteration: 300 loss: 0.0084 lr: 0.005
iteration: 400 loss: 0.0067 lr: 0.005
iteration: 400 loss: 0.0067 lr: 0.005
iteration: 400 loss: 0.0067 lr: 0.005
iteration: 500 loss: 0.0068 lr: 0.005
iteration: 500 loss: 0.0068 lr: 0.005
iteration: 500 loss: 0.0068 lr: 0.005
iteration: 600 loss: 0.0050 lr: 0.005
iteration: 600 loss: 0.0050 lr: 0.005
iteration: 600 loss: 0.0050 lr: 0.005
iteration: 700 loss: 0.0044 lr: 0.005
iteration: 700 loss: 0.0044 lr: 0.005
iteration: 700 loss: 0.0044 lr: 0.005
iteration: 800 loss: 0.0044 lr: 0.005
iteration: 800 loss: 0.0044 lr: 0.005
iteration: 800 loss: 0.0044 lr: 0.005
iteration: 900 loss: 0.0046 lr: 0.005
iteration: 900 loss: 0.0046 lr: 0.005
iteration: 9

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [9]:
deeplabcut.evaluate_network(path_config_file)

Config:
{'all_joints': [[0]],
 'all_joints_names': ['Separator'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_FlyFeb20/Fly_Sayed95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_FlyFeb20/Documentation_data-Fly_95shuffle1.pickle',
 'minsize': 100,
 'mirror': False,
 'multi_step': [[0.005, 10000],
                

Running  DeepCut_resnet50_FlyFeb20shuffle1_200  with # of trainingiterations: 200
INFO:tensorflow:Restoring parameters from /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Fly-Sayed-2019-02-20/dlc-models/iteration-0/FlyFeb20-trainset95shuffle1/train/snapshot-200


Restoring parameters from /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Fly-Sayed-2019-02-20/dlc-models/iteration-0/FlyFeb20-trainset95shuffle1/train/snapshot-200
Restoring parameters from /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Fly-Sayed-2019-02-20/dlc-models/iteration-0/FlyFeb20-trainset95shuffle1/train/snapshot-200
0it [00:00, ?it/s]

Analyzing data...


20it [00:26,  1.30s/it]


Done and results stored for snapshot:  snapshot-200
Results for 200  training iterations: 95 1 train error: 308.79 pixels. Test error: 308.4  pixels.
With pcutoff of 0.1  train error: 308.79 pixels. Test error: 308.4 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [7]:
videofile_path = ["/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_top.avi",
                  "/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_bottom.avi"] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

Config:
{'all_joints': [[0]],
 'all_joints_names': ['Fly1'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Track_fliesFeb20/Track_flies_Sayed95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Track_fliesFeb20/Documentation_data-Track_flies_95shuffle1.pickle',
 'minsize': 100,
 'mirror': False,
 'multi_step': [[0.00

Using snapshot-200 for model /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/dlc-models/iteration-0/Track_fliesFeb20-trainset95shuffle1
INFO:tensorflow:Restoring parameters from /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/dlc-models/iteration-0/Track_fliesFeb20-trainset95shuffle1/train/snapshot-200


Restoring parameters from /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Track_flies-Sayed-2019-02-20/dlc-models/iteration-0/Track_fliesFeb20-trainset95shuffle1/train/snapshot-200
  0%|          | 0/223 [00:00<?, ?it/s]

Starting to analyze %  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_top.avi
Loading  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_top.avi
Duration of video [s]:  22.3 , recorded with  10.0 fps!
Overall # of frames:  223  found with (before cropping) frame dimensions:  832 372
Starting to extract posture


230it [02:17,  1.55it/s]                         

Detected frames:  223



  0%|          | 0/223 [00:00<?, ?it/s]

Saving results in /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos...
Starting to analyze %  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_bottom.avi
Loading  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_bottom.avi
Duration of video [s]:  22.3 , recorded with  10.0 fps!
Overall # of frames:  223  found with (before cropping) frame dimensions:  832 372
Starting to extract posture


230it [02:17,  1.54it/s]                         

Detected frames:  223
Saving results in /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi'])

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [7]:
%gui wx
deeplabcut.refine_labels(path_config_file)

ERROR:root:Cannot activate multiple GUI eventloops


/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Moving-Antonio-2019-02-19/labeled-data/moving_hand
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [8]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

 19%|█▉        | 42/223 [00:00<00:00, 414.88it/s]

Starting %  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos ['/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_top.avi', '/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_bottom.avi']
Loading  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_top.avi and data.
False 0 832 0 372
223
Duration of video [s]:  22.3 , recorded with  10.0 fps!
Overall # of frames:  223 with cropped frame dimensions:  832 372
Generating frames and creating video.


 19%|█▉        | 42/223 [00:00<00:00, 411.29it/s]

Starting %  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos ['/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_top.avi', '/home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_bottom.avi']
Loading  /home/antortjim/MEGA/FlySleepLab/FlyRecognition/videos/main_videos/outpy05_reshaped_bottom.avi and data.
False 0 832 0 372
223
Duration of video [s]:  22.3 , recorded with  10.0 fps!
Overall # of frames:  223 with cropped frame dimensions:  832 372
Generating frames and creating video.


100%|██████████| 223/223 [00:00<00:00, 393.65it/s]


## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)